In [ ]:
# Installation of FEniCS in Colab via
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

### Time dependent Stokes equation ###

In [ ]:
from fenics import *
from mshr import *
from pylab import plt
import numpy as np

# generate domains and meshes

L  = 23.0
H  = 12.0
X  = 3.0
r  = 0.5
nr = 8*16 # 48

rect   = Polygon([Point(0,0),Point(L,0),Point(L,H),Point(0,H)])
disc   = Circle(Point(X,H/2),r,nr)
domain = rect - disc
mesh   = generate_mesh(domain,64)

def create_markers(mesh,X0,xr,yr):
    cell_markers = MeshFunction("bool",mesh,mesh.geometric_dimension())
    for cell in cells(mesh):
        cell_index = cell.index()
        pos = np.reshape(cell.get_vertex_coordinates(),(-1,2))
        xm  = np.mean(pos[:,0])
        ym  = np.mean(pos[:,1])
        rr = ((xm-X0)**2)/(xr*xr) + ((ym-H/2)**2)/(yr*yr)
        if (rr < 1):
            cell_markers[cell_index] = True
        else:
            cell_markers[cell_index] = False
    return cell_markers

def update_particles(xpart,u):
    u.set_allow_extrapolation(True)
    for k in range(npart):
        uu = u(xpart[k,0],xpart[k,1])
        xpart[k,0] += tau*uu[0]
        xpart[k,1] += tau*uu[1]
        if xpart[k,0] > L:
            xpart[k,0] -= L
    return xpart
    
def outvtk_particles(fname,xpart):
    ff = open(fname + '.vtk', 'w')
    ff.write('# vtk DataFile Version 2.0\n')
    ff.write('Particle positions\n')
    ff.write('ASCII\n')
    ff.write('DATASET POLYDATA\n')
    npoint = np.size(xpart,0)
    ff.write('POINTS '+str(npoint)+' float\n')
    for i in range(0,npoint):
        ff.write(str(xpart[i,0])+' '+str(xpart[i,1])+' 0\n')
    ff.close()  


mesh = refine(mesh,create_markers(mesh,  8, 6.0, 3.0))
mesh = refine(mesh,create_markers(mesh,  6, 4.0, 1.5))
mesh = refine(mesh,create_markers(mesh,  4, 2.0, 1.0))
# mesh = refine(mesh,create_markers(mesh,  3, 0.6, 0.6))

plt.figure(figsize=(20,20))
plot(mesh,linewidth=0.5)
plt.xlim([0,10])

print(mesh.num_vertices())

### Example 4: Convection-diffusion problem ###

We solve the time-dependent problem




\begin{align}
\int_\Omega \partial_t q v\mathrm{d}x + a(q,v)=0
\end{align}

where

$$
a(q,v)=\int_\Omega \alpha\nabla q\cdot\nabla v + (\vec{u}\cdot\nabla q)v\mathrm{d}x
$$

for given functions $\alpha,\vec{u}$.

In [ ]:
# FE definitions
parameters["form_compiler"]["quadrature_degree"] = 3

V    = VectorFunctionSpace(mesh,"CG",1)
W    = FunctionSpace(mesh,"CG",1)

# Define Dirichlet boundary conditions
tol     = 1E-4
def boundary_dirichlet(x, on_boundary):
    return on_boundary and (near(x[1], 0, tol) or near(x[1], H, tol) or near(x[0], 0, tol))

def boundary_disc(x, on_boundary):
    r2 = (x[0]-X)**2 + (x[1]-H/2)**2
    return on_boundary  and near(r2,r*r,0.02)

def boundary_outflow(x, on_boundary):
    return on_boundary and near(x[0], L, tol)


ubc = Expression(("1.0","0"),H=H,degree=3)
bc1 = DirichletBC(V, ubc, boundary_dirichlet)
bc2 = DirichletBC(V, Constant((0.0,0.0)), boundary_disc)
bc  = [bc1,bc2]

bco = DirichletBC(W, Constant(0.0), boundary_outflow)

# single time step
def evolve(old_u, tau):
    u,du = TrialFunction(V),TestFunction(V) 
    
    #u,du = Function(V),TestFunction(V) 
    #dudt = (u - old_u)/tau
    #Res  = inner(dudt,du)*dx 
    #Res += inner(dot(grad(old_u),old_u),du)*dx
    #Res += mu*inner(grad(u),grad(du))*dx
    #solve(Res==0, u, bc) 
    
    a   = inner(u/tau,du)*dx + mu*inner(grad(u),grad(du))*dx
    L   = inner(old_u/tau,du)*dx - inner(dot(grad(old_u),old_u),du)*dx
    
    u = Function(V)
    u.assign(old_u)
    solve(a==L, u, bc) 

    return u

# single time step
def chorin_project(old_u):
    
    phi,dphi        = TrialFunction(W),TestFunction(W) 

    a   = inner(grad(phi),grad(dphi))*dx
    L   = div(old_u)*dphi*dx  
    
    phi = Function(W)
    solve(a==L, phi,bco) 
    u   = project(old_u + grad(phi),V)

    return u 
    
# main time loop
mu      =  1/150
n_steps =  5000
tau     =  0.002 # 025
t       =  0.0

ndim  = 2
npart = 500
for k in range(npart):
        while True:
            xp = np.random.rand()*L
            yp = np.random.rand()*H
            if (((xp-X)**2+(yp-H/2)**2)>(r*r)):
                break
        xpart[k,0] = xp
        xpart[k,1] = yp

old_u   = interpolate(Expression(("1.0","0.5*sin(x[0])"),degree=2),V)
file_u  = File("output/lecture10_u.pvd")
file_w  = File("output/lecture10_w.pvd")
mprint  = 0
for i in range(n_steps):
    print(i)
    t = t + tau
    u_int = evolve(old_u, tau)
    u     = chorin_project(u_int)

    xpart = update_particles(xpart,u)
    
    if ((i % 10) == 0):
        outvtk_particles('output/lecture10_part'+str(mprint),xpart)
        ucurl = u[1].dx(0)-u[0].dx(1)
        w = project(ucurl,W)
        u.rename("u","u")
        w.rename("w","w")
        file_u << (u,t)
        file_w << (w,t)
        
        mprint += 1
        
    old_u.assign(u)